In [1]:
from xml.etree import ElementTree as ET
import folium
import numpy as np
from shapely.geometry import Point, LineString
import pandas as pd
from sqlalchemy import create_engine
from math import radians, cos, sin, sqrt, atan2, asin
import math
import ast
import json

/home/marcelo-borges/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dbname = 'poc-db'
user = 'adminpoc'
password = 'adminpoc'
host = 'localhost'
port = '5440' 

In [3]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [4]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

In [5]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

In [6]:
sql_idle_route = """ 
SELECT "data", carro, first_travel, last_travel, geolocalizacao_ida, geolocalizacao_volta
FROM hp.gld_idle_route;
"""

In [7]:
sql_schedule = """ 
SELECT date_execution, linha, carro, re, nome
FROM hp.slv_schedule;
"""

In [8]:
df_idle_route = pd.read_sql_query(sql_idle_route, engine)

In [9]:
df_idle_route

,data,carro,first_travel,last_travel,geolocalizacao_ida,geolocalizacao_volta
0,2024-03-12,20442,603A,603B,"[{'lat': -16.76350454639546, 'lng': -49.270226...","[{'lat': -16.824009382344, 'lng': -49.30804048..."
1,2024-03-13,20318,574A,574A,"[{'lat': -16.70513240150447, 'lng': -49.300648...","[{'lat': -16.70948506823616, 'lng': -49.309943..."
2,2024-03-08,20324,026A,026C,"[{'lat': -16.70513240150447, 'lng': -49.300648...","[{'lat': -16.70948506823616, 'lng': -49.309943..."
3,2024-03-20,20434,006B,010A,"[{'lat': -16.76350454639546, 'lng': -49.270226...","[{'lat': -16.78750134432972, 'lng': -49.278417..."
4,2024-04-16,20308,028A,281A,"[{'lat': -16.70513240150447, 'lng': -49.300648...","[{'lat': -16.70948506823616, 'lng': -49.309943..."
...,...,...,...,...,...,...
13481,2024-04-03,20488,213C,213B,"[{'lat': -16.76350421438977, 'lng': -49.270228...","[{'lat': -16.81183465185691, 'lng': -49.241870..."
13482,2024-04-15,20362,400A,400A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794..."
13483,2024-03-25,20432,010B,006C,"[{'lat': -16.76350454639546, 'lng': -49.270226...","[{'lat': -16.78750134432972, 'lng': -49.278417..."
13484,2024-03-04,20485,002A,208A,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...","[{'lat': -16.74761406304063, 'lng': -49.205439..."


In [10]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [11]:
placemarks_line_bd = read_and_extract_placemarks(planned_trip_path, ns)

In [12]:
df_base_rota = pd.DataFrame(placemarks_line_bd)

In [13]:
df_base_rota['Código'] = df_base_rota['name'].apply(lambda x: x.split('-', 1)[0])

In [14]:
df_base_rota['identificacao'] = df_base_rota['name'].apply(lambda x: x.split('-', 1)[1])
df_base_rota['identificacao'] = df_base_rota['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_base_rota['identificacao'] = df_base_rota['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [15]:
df_base_rota['Código'] = df_base_rota['name'].apply(lambda x: x.split('-', 1)[0])
df_base_rota['Descrição e Sentido'] = df_base_rota['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_base_rota['Descrição'] = df_base_rota['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_base_rota['Sentido'] = df_base_rota['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_base_rota.drop('Descrição e Sentido', axis=1, inplace=True)

In [16]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [17]:
df_base_rota['sublinha'] = df_base_rota['identificacao'].apply(lambda x: de_para[x[:-1]])

In [18]:
df_base_rota['num_linha'] = df_base_rota['Código'].astype(str) + df_base_rota['sublinha']

In [227]:
# Substituir todos os valores "401A" por "401" na coluna "num_linha" do DataFrame df_base_rota
df_base_rota['num_linha'] = df_base_rota['num_linha'].replace('401A', '401')

In [20]:
df_schedule = pd.read_sql_query(sql_schedule, engine)

In [21]:
df_schedule['date_execution'] = pd.to_datetime(df_schedule['date_execution']) 

In [143]:
df_schedule['linha']=df_schedule['linha'].str.strip()

In [144]:
df_schedule[df_schedule['date_execution'] == '2024-04-15'][['linha', 'carro', 're', 'nome']]

,linha,carro,re,nome
180,117A,1201,000007200,LINDOMAR ANTONIO GARCIA
182,117A,1201,000007200,LINDOMAR ANTONIO GARCIA
184,117A,1201,000007200,LINDOMAR ANTONIO GARCIA
186,117A,1201,000018995,RICARDO ALEXANDRE CHAVES
188,117A,1201,000018995,RICARDO ALEXANDRE CHAVES
...,...,...,...,...
92855,026A,20998,000018886,EZEQUIEL DA SILVA HUTIM
92856,028A,20998,000017239,RONES SOARES DOS SANTOS
92857,027A,20998,000017239,RONES SOARES DOS SANTOS
92858,028A,20998,000017239,RONES SOARES DOS SANTOS


In [145]:
list_car = df_schedule['carro'].unique().tolist()

In [146]:
sql_made_trip = """ 
SELECT registration, carro, made_trip_coordinates, execution_date
FROM hp.slv_made_trip;

"""

In [147]:
df_made_trip = pd.read_sql_query(sql_made_trip,engine)

In [148]:
df_idle_route['data'] = pd.to_datetime(df_idle_route['data'])

In [149]:
df_made_trip['execution_date'] = pd.to_datetime(df_made_trip['execution_date'])

In [150]:
df_made_trip['carro'] = pd.to_numeric(df_made_trip['carro'])

In [151]:
df_test = pd.merge(df_idle_route, df_made_trip, left_on=['data','carro'], right_on=['execution_date','carro'])

In [152]:
list_line = df_schedule[(df_schedule['carro'] == 20018) & (df_schedule['date_execution'] == '2024-04-15')]['linha'].values.tolist()

In [153]:
list_line

['015A',
 '015A',
 '176A',
 '160A',
 '176A',
 '400A',
 '160A',
 '015A',
 '015A',
 '176A',
 '160A',
 '176A',
 '400A',
 '160A']

In [154]:
df_base_rota[df_base_rota['num_linha']=='176A']

,name,coordinates,Código,identificacao,Descrição,Sentido,sublinha,num_linha
316,176-1i - T. PRACA A / CAIS CAMPINAS / SANTA HE...,"-49.284614,-16.673452,0 -49.284557997,-16.6734...",176,1i,T. PRACA A / CAIS CAMPINAS / SANTA HELENA,IDA,A,176A
317,176-1v - T. PRACA A / CAIS CAMPINAS / SANTA HE...,"-49.283947481,-16.664584563,0 -49.283864278,-1...",176,1v,T. PRACA A / CAIS CAMPINAS / SANTA HELENA,VOLTA,A,176A


In [155]:
df_filtered = pd.merge(df_idle_route, df_made_trip, left_on=['data','carro'], right_on=['execution_date','carro'])

In [156]:
df_teste = df_filtered[df_filtered['carro'] == 20018]

In [157]:
def convert_coordinates(coords):
    return [(lat, lng) for [lng, lat] in coords]

In [158]:
def parse_coordinates(coord_string):
    # Remover possíveis espaços em branco extra
    coord_string = coord_string.strip()
    
    # Dividir a string por espaço para separar os pares de coordenadas
    coords = coord_string.split(' ')
    
    # Dividir cada par por vírgula e converter para float
    parsed_coords = []
    for point in coords:
        if point:  # Verifica se a string não está vazia
            try:
                components = point.split(',')
                if len(components) >= 2:  # Certifica que existem pelo menos dois valores (lat, lng)
                    lat, lng = components[:2]  # Usa apenas os dois primeiros valores
                    parsed_coords.append((float(lat), float(lng)))
            except ValueError as e:
                print(f"Erro ao processar o ponto '{point}': {e}")
                continue  # Pula para o próximo ponto
    
    return parsed_coords

In [159]:
list_car = df_made_trip[df_made_trip['execution_date'] == '2024-04-15']['carro'].unique().tolist()

In [160]:
df_filtered = pd.merge(df_idle_route, df_made_trip, left_on=['data','carro'], right_on=['execution_date','carro'])

In [161]:
df_filtered.dtypes

data                     datetime64[ns]
carro                             int64
first_travel                     object
last_travel                      object
geolocalizacao_ida               object
geolocalizacao_volta             object
registration                     object
made_trip_coordinates            object
execution_date           datetime64[ns]
dtype: object

In [162]:
df_filtered[df_filtered['data']=='2024-04-15']

,data,carro,first_travel,last_travel,geolocalizacao_ida,geolocalizacao_volta,registration,made_trip_coordinates,execution_date
26,2024-04-15,20048,401,401,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",PQA2792,"[[-49.362301, -16.81122], [-49.362246, -16.811...",2024-04-15
45,2024-04-15,20224,400A,911A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",NKE9854,"[[-49.301699, -16.704075], [-49.301809, -16.70...",2024-04-15
145,2024-04-15,20357,176A,015A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",NLC7694,"[[-49.301395, -16.702537], [-49.301461, -16.70...",2024-04-15
235,2024-04-15,20021,401,403B,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.70948506823616, 'lng': -49.309943...",ONQ9479,"[[-49.301364, -16.704743]]",2024-04-15
247,2024-04-15,20360,401,401,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",NJY3685,"[[-49.301201, -16.703086], [-49.301153, -16.70...",2024-04-15
348,2024-04-15,20238,015A,400A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",NKC3574,"[[-49.301015, -16.704094], [-49.301215, -16.70...",2024-04-15
368,2024-04-15,20019,915A,400A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",ONW2749,"[[-49.301651, -16.704151], [-49.301638, -16.70...",2024-04-15
394,2024-04-15,20805,025A,025A,"[{'lat': -16.70513240150447, 'lng': -49.300648...","[{'lat': -16.70948506823616, 'lng': -49.309943...",SCQ3A44,"[[-49.301633, -16.705014], [-49.301468, -16.70...",2024-04-15
400,2024-04-15,20018,015A,160A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",ONU6399,"[[-49.301646, -16.704247], [-49.30138, -16.703...",2024-04-15
401,2024-04-15,20362,400A,400A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",NLB6B14,"[[-49.301187, -16.702887], [-49.301255, -16.70...",2024-04-15


In [163]:
df_filtered[(df_filtered['data'] == '2024-04-15') & (df_filtered['carro'] == 20018)]

,data,carro,first_travel,last_travel,geolocalizacao_ida,geolocalizacao_volta,registration,made_trip_coordinates,execution_date
400,2024-04-15,20018,015A,160A,"[{'lat': -16.705151223923, 'lng': -49.30067673...","[{'lat': -16.673411078397, 'lng': -49.28460794...",ONU6399,"[[-49.301646, -16.704247], [-49.30138, -16.703...",2024-04-15


In [164]:
TOLERANCIA = 2.5

In [165]:
DISTANCIA_TOLERANCIA = 1 #km 

In [166]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees).
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

def find_closest_point(real_point, planned_route):
    min_distance = float('inf')
    closest_point = None
    for planned_point in planned_route:
        # Ensure the longitude comes first in the haversine function
        distance = haversine(planned_point['lng'], planned_point['lat'], real_point[1], real_point[0])
        if distance < min_distance:
            min_distance = distance
            closest_point = planned_point
    return min_distance, closest_point

def find_closest_point_and_check_tolerance(real_point, planned_route, tolerance=TOLERANCIA):
    min_distance, closest_point = find_closest_point(real_point, planned_route)
    return min_distance, closest_point, min_distance > tolerance


In [168]:
distancia = haversine(lon1=-49.289994, lat1=-16.659657, lon2=-49.290064, lat2=-16.659582)
print(f"A distância entre os pontos é {distancia} km")

A distância entre os pontos é 0.011187268604866242 km


In [170]:
df_idle_route[(df_idle_route['data']=='2024-04-15')&(df_idle_route['carro']==20018)].to_csv('./idle_route.csv', index=False)

In [171]:
df_made_trip[(df_made_trip['execution_date']=='2024-04-15')&(df_made_trip['carro']==20018)].to_csv('./made_trip.csv', index=False)

In [172]:
df_schedule[(df_schedule['date_execution']=='2024-04-15')&(df_schedule['carro']==20018)].to_csv('./schedule.csv', index=False)

In [175]:
df_base_rota.to_csv('./rotas_programadas.csv')

In [272]:
import folium
import pandas as pd
import math
import json
from folium.plugins import FloatImage

# Função para calcular a distância entre dois pontos usando a fórmula de Haversine
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Função para converter string JSON em coordenadas de latitude e longitude
def parse_coordinates_format_dict(coord_data):
    """
    Função para coordenadas no formato de lista de dicionários: [{'lat': lat, 'lng': lng}, ...]
    """
    try:
        result = [(coord['lat'], coord['lng']) for coord in coord_data]
    except (KeyError, ValueError):
        result = []
    return result

def parse_coordinates_format_list(coord_data):
    """
    Função para coordenadas no formato de lista de listas: [[lng, lat], ...]
    """
    try:
        result = [(coord[1], coord[0]) for coord in coord_data]
    except (IndexError, ValueError):
        result = []
    return result

def parse_coordinates_format_str(coord_data):
    """
    Função para coordenadas no formato de string: lng, lat
    """
    try:
        coords = [coord.split(',') for coord in coord_data]
        result = [(float(coord[1]), float(coord[0])) for coord in coords]
    except (IndexError, ValueError):
        result = []
    return result

def parse_coordinates_format_text(coord_data):
    """
    Função para coordenadas no formato de texto: "lng,lat,alt lng,lat,alt ..."
    """
    try:
        # Dividir a string de coordenadas em uma lista de strings separadas por espaço
        coords_list = coord_data.split(' ')
        # Cada coordenada é uma string no formato 'lng,lat,alt'
        # Vamos dividir isso em uma lista de listas onde cada sublista contém [lng, lat]
        parsed_coords = [[float(coord.split(',')[1]), float(coord.split(',')[0])] for coord in coords_list]
        return parsed_coords
    except (IndexError, ValueError):
        return []



# Função principal para processar cada carro
# Função para processar cada carro
# Função principal para processar cada carro
def process_car(df_idle_route, df_made_trip, df_schedule, df_base_rota, car):
    df_filtered = pd.merge(df_idle_route, df_made_trip, left_on=['data', 'carro'], right_on=['execution_date', 'carro'])
    df_filtered = df_filtered[(df_filtered['data'] == '2024-04-15') & (df_filtered['carro'] == car)]

    if df_filtered.empty:
        print(f"Nenhum dado para o carro {car} no dia especificado.")
        return

    # Inicializar um mapa
    first_row = df_filtered.iloc[0]
    lat_start = (first_row['geolocalizacao_ida'][0]['lat'] + first_row['geolocalizacao_volta'][0]['lat']) / 2
    lng_start = (first_row['geolocalizacao_ida'][0]['lng'] + first_row['geolocalizacao_volta'][0]['lng']) / 2
    m = folium.Map(location=[lat_start, lng_start], zoom_start=12)

    # Plotar rotas ociosas
    ida = parse_coordinates_format_dict(first_row['geolocalizacao_ida'])
    volta = parse_coordinates_format_dict(first_row['geolocalizacao_volta'])
    folium.PolyLine(ida, color='blue', weight=2.5, opacity=1, tooltip='Rota Ociosa - Ida').add_to(m)
    folium.PolyLine(volta, color='red', weight=2.5, opacity=1, tooltip='Rota Ociosa - Volta').add_to(m)

    # Plotar a rota realizada
    made_trip = parse_coordinates_format_list(first_row['made_trip_coordinates'])
    folium.PolyLine(made_trip, color='green', weight=2.5, opacity=1, tooltip='Rota Realizada').add_to(m)

    # Para cada linha planejada, plotar a rota correspondente
    list_line = df_schedule[(df_schedule['carro'] == car) & (df_schedule['date_execution'] == '2024-04-15')]['linha'].values.tolist()
    for line in list_line:
        print(line)
        df_base_rota_line = df_base_rota[df_base_rota['num_linha'] == line]
        coordinates = df_base_rota_line['coordinates'].iloc[0]
        if isinstance(coordinates, str):
            parsed_coordinates = parse_coordinates_format_text(coordinates)
        else:
            parsed_coordinates = parse_coordinates_format_dict(coordinates)
        
        # Plotar a rota planejada
        folium.PolyLine(parsed_coordinates, color='gray', weight=2.5, opacity=0.5, dash_array='5, 5',tooltip=f'Linha {line}').add_to(m)

    # Calcular distâncias
    planned_distance = sum([haversine(ida[i][1], ida[i][0], ida[i+1][1], ida[i+1][0]) for i in range(len(ida)-1)])
    planned_distance += sum([haversine(volta[i][1], volta[i][0], volta[i+1][1], volta[i+1][0]) for i in range(len(volta)-1)])
    for route in list_line:
        df_base_rota_line = df_base_rota[df_base_rota['num_linha'] == route]
        coordinates = df_base_rota_line['coordinates'].iloc[0]
        if isinstance(coordinates, str):
            route_coords = parse_coordinates_format_text(coordinates)
        else:
            route_coords = parse_coordinates_format_dict(coordinates)
        planned_distance += sum([haversine(route_coords[i][1], route_coords[i][0], route_coords[i+1][1], route_coords[i+1][0]) for i in range(len(route_coords)-1)])

    real_distance = sum([haversine(made_trip[i][1], made_trip[i][0], made_trip[i+1][1], made_trip[i+1][0]) for i in range(len(made_trip)-1)])
    
    # Adicionar informações sobre a distância percorrida
    info_html = f"""
    <div style="position: fixed; 
                 bottom: 50px; right: 50px; width: 300px; height: 100px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">&nbsp; <b>Informações de Distância:</b> <br>
                   &nbsp; Distância Planejada: {planned_distance:.2f} km <br>
                   &nbsp; Distância Realizada: {real_distance:.2f} km <br>
                   &nbsp; Diferença de Distância: {planned_distance - real_distance:.2f} km <br>
    </div>
    """
    m.get_root().html.add_child(folium.Element(info_html))

    # Salvar o mapa em um arquivo HTML
    mapa_filename = f'map_{car}.html'
    m.save(mapa_filename)
    print(f"Mapa salvo como {mapa_filename}")


# Exemplo de uso da função process_car
# Você precisará fornecer os DataFrames df_idle_route, df_made_trip, df_schedule e o identificador do carro
# process_car(df_idle_route, df_made_trip, df_schedule,df_base_rota, car_id)

In [273]:
for car in list_car:
    print(car)
    process_car(df_idle_route, df_made_trip, df_schedule, df_base_rota, car)

20805
025A
025A
025A
025A
025A
025A
025A
025A
025C
025C
025A
025A
025A
025A
025A
025A
025A
025A
025A
025A
025A
025C
025C
025A
025A
025A
Mapa salvo como map_20805.html
20238
015A
400A
176A
401
911B
400A
015A
400A
176A
401
911B
400A
Mapa salvo como map_20238.html
20018
015A
015A
176A
160A
176A
400A
160A
015A
015A
176A
160A
176A
400A
160A
Mapa salvo como map_20018.html
20360
401
400A
015A
400A
401
911B
400A
401
401
401
400A
015A
400A
401
911B
400A
401
401
Mapa salvo como map_20360.html
20362
400A
911A
400A
176A
015A
023A
400A
400A
911A
400A
176A
015A
023A
400A
Mapa salvo como map_20362.html
20021
401
400A
911A
400A
401
911B
911B
160A
403B
401
400A
911A
400A
401
911B
911B
160A
403B
Mapa salvo como map_20021.html
20224
400A
911A
911A
400A
911A
911A
Mapa salvo como map_20224.html
20048
401
400A
176A
160A
400A
400A
401
401
400A
176A
160A
400A
400A
401
Mapa salvo como map_20048.html
20019
915A
401
401
401
401
400A
915A
401
401
401
401
400A
Mapa salvo como map_20019.html
20357
176A
160A
015A
91